In [5]:
import pandas as pd
from pathlib import Path
import numpy as np

input_dir = Path("res")
input_dir.mkdir(exist_ok=True)

csv_files = list(input_dir.glob("*.csv"))

if not csv_files:
    print("Aucun fichier CSV trouvé dans le dossier 'res'.")
else:
    for csv_path in csv_files:
        try:
            df = pd.read_csv(csv_path)

            # Vérifs basiques
            if "locality" not in df.columns or "trend_score" not in df.columns:
                print(f"⚠️ {csv_path.name} doit contenir les colonnes 'locality' et 'trend_score'. Ignoré.")
                continue

            # Forcer numérique (si jamais il y a des strings), conserver NaN pour éviter plantage
            df["trend_score"] = pd.to_numeric(df["trend_score"], errors="coerce")

            # Séparer par signe
            pos_mask = df["trend_score"] > 0
            neg_mask = df["trend_score"] < 0
            zero_mask = df["trend_score"] == 0

            # Max positif et min négatif (le plus petit, donc le plus négatif)
            max_pos = df.loc[pos_mask, "trend_score"].max() if pos_mask.any() else None
            min_neg = df.loc[neg_mask, "trend_score"].min() if neg_mask.any() else None  # ex: -18.16

            # Créer une copie pour travailler proprement
            ts = df["trend_score"].copy()

            # Positifs → [0, 10] (si au moins un positif et max_pos != 0)
            if max_pos is not None and max_pos != 0:
                ts.loc[pos_mask] = 10 * df.loc[pos_mask, "trend_score"] / max_pos

            # Négatifs → [-10, 0] (si au moins un négatif et min_neg != 0)
            if min_neg is not None and min_neg != 0:
                # min_neg est négatif, on obtient bien -10 au min, proche de 0 pour faibles valeurs
                ts.loc[neg_mask] = -10 * df.loc[neg_mask, "trend_score"] / min_neg

            # Zéros inchangés
            ts.loc[zero_mask] = 0

            # Si un côté n’existe pas (ex. que des positifs), on laisse l’autre côté tel quel.
            # Cas limites: si max_pos==0 (tous 0) ou min_neg==0 (n’arrive pas logiquement), tout reste 0.

            # Remplacer la colonne trend_score par la version normalisée
            df["trend_score"] = ts

            # Ne garder que les deux colonnes demandées et dans le bon ordre
            df_out = df[["locality", "trend_score"]]

            # Enregistrement
            output_path = csv_path.with_name(csv_path.stem + "_norma.csv")
            df_out.to_csv(output_path, index=False)
            print(f"✅ Normalisé (signe préservé) → {output_path.name}")

        except Exception as e:
            print(f"❌ Erreur avec {csv_path.name} : {e}")


✅ Normalisé (signe préservé) → restau_trend_score_norma.csv
✅ Normalisé (signe préservé) → third_sector_establishment_score_by_locality_norma.csv
✅ Normalisé (signe préservé) → third_sector_job_score_by_locality_norma.csv
✅ Normalisé (signe préservé) → building_trend_norma.csv
✅ Normalisé (signe préservé) → demo_trend_score_norma.csv
✅ Normalisé (signe préservé) → trend_stations_traffic_2_norma.csv


# score tot

In [4]:
import polars as pl
import pandas as pd

In [10]:
# Chemins des fichiers
files = {
    'third_sector_job': 'res/third_sector_job_score_by_locality_norma.csv',
    'building': 'res/building_trend_norma.csv',
    'demographie': 'res/demo_trend_score_norma.csv',
    'restau': 'res/restau_trend_score_norma.csv',
    'third_sector_establishment': 'res/third_sector_establishment_score_by_locality_norma.csv'
}

# Charger tous les fichiers
df_third_sector_job = pl.read_csv(files['third_sector_job']).rename({'trend_score': 'third_sector_job_score'})
df_building = pl.read_csv(files['building']).rename({'trend_score': 'building_score'})
df_demo = pl.read_csv(files['demographie']).rename({'trend_score': 'demographie_score'})
df_restau = pl.read_csv(files['restau']).rename({'trend_score': 'restau_score'})
df_establishment = pl.read_csv(files['third_sector_establishment']).rename({'trend_score': 'third_sector_establishment_score'})

print("Fichiers chargés:")
print(f"  • Third Sector Jobs: {df_third_sector_job.shape[0]} communes")
print(f"  • Building: {df_building.shape[0]} communes")
print(f"  • Démographie: {df_demo.shape[0]} communes")
print(f"  • Restaurants: {df_restau.shape[0]} communes")
print(f"  • Third Sector Establishments: {df_establishment.shape[0]} communes")

# Fusionner progressivement avec coalesce=True pour éviter les duplications
df_final = df_third_sector_job.join(
    df_building, 
    on='locality', 
    how='full',
    coalesce=True
)
print(f"\nAprès fusion building: {df_final.shape[0]} communes")

df_final = df_final.join(
    df_demo, 
    on='locality', 
    how='full',
    coalesce=True
)
print(f"Après fusion démographie: {df_final.shape[0]} communes")

df_final = df_final.join(
    df_restau, 
    on='locality', 
    how='full',
    coalesce=True
)
print(f"Après fusion restaurant: {df_final.shape[0]} communes")

df_final = df_final.join(
    df_establishment, 
    on='locality', 
    how='full',
    coalesce=True
)
print(f"Après fusion établissements tertiaire: {df_final.shape[0]} communes")

print(f"\n✅ Toutes les données fusionnées!")
print(f"Total de communes: {df_final.shape[0]}")
print(f"\nAperçu:")
print(df_final.head(10))

Fichiers chargés:
  • Third Sector Jobs: 2104 communes
  • Building: 2136 communes
  • Démographie: 2131 communes
  • Restaurants: 1745 communes
  • Third Sector Establishments: 2089 communes

Après fusion building: 2136 communes
Après fusion démographie: 2137 communes
Après fusion restaurant: 2143 communes
Après fusion établissements tertiaire: 2143 communes

✅ Toutes les données fusionnées!
Total de communes: 2143

Aperçu:
shape: (10, 6)
┌────────────────┬────────────────┬────────────────┬────────────────┬──────────────┬───────────────┐
│ locality       ┆ third_sector_j ┆ building_score ┆ demographie_sc ┆ restau_score ┆ third_sector_ │
│ ---            ┆ ob_score       ┆ ---            ┆ ore            ┆ ---          ┆ establishment │
│ str            ┆ ---            ┆ f64            ┆ ---            ┆ f64          ┆ _sco…         │
│                ┆ f64            ┆                ┆ f64            ┆              ┆ ---           │
│                ┆                ┆                

In [ ]:
# Remplacer les valeurs null par 0 pour le calcul du total
df_final = df_final.with_columns([
    pl.col('third_sector_job_score').fill_null(0),
    pl.col('building_score').fill_null(0),
    pl.col('demographie_score').fill_null(0),
    pl.col('restau_score').fill_null(0),
    pl.col('third_sector_establishment_score').fill_null(0)
])

# Calculer le score total (somme de tous les scores)
df_final = df_final.with_columns(
    (
        pl.col('third_sector_job_score') +
        pl.col('building_score') +
        pl.col('demographie_score') +
        pl.col('restau_score') +
        pl.col('third_sector_establishment_score')
    ).alias('total')
)

# Trier par score total décroissant
df_final = df_final.sort('total', descending=True)

print("✅ Score total calculé!")
print(f"\nAperçu du résultat final:")
df_final.head(10)